## Imports

In [ ]:

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('__file__'), '..', 'src')))

#%pip install -r requirements.txt
from data_preprocessing import data_preprocessing_tumor
from model import BrainCNN, EarlyStopping


import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchbearer
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torchbearer import Trial
from torchmetrics import Precision, Recall, F1Score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score



In [ ]:

""" 
first try paper CCE, then NLLLoss
cce_loss_fn = nn.NLLLoss()
cce_logits_loss_fn = nn.CrossEntropyLoss()



>>> optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)
>>> optimizer.zero_grad()
>>> loss_fn(model(input), target).backward()
>>> optimizer.step()
    def train(model, train_loader, valid_loader, ):

"""



""" 
important

2 combinations should be tested:
1. last layer Softmax + loss func nn.NLLLoss
2. no last layer and CrossEntropyLoss
"""


train_loader, valid_loader, test_loader = data_preprocessing_tumor()
num_epochs=100
patience=1
learning_rate=0.001
device=None
model = BrainCNN()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
early_stopping = EarlyStopping(patience=5, delta=0.01)

device = "cuda:0" if torch.cuda.is_available() else "cpu"


    
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        
    train_loss /= len(train_loader.dataset)
    
    model.eval()
    
    val_loss = 0.0
    with torch.no_grad():
        for data, target in valid_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            val_loss += loss.item() * data.size(0)
            
    val_loss /= len(valid_loader.dataset) 
    
    print(f"Epoch: {epoch+1}, Train loss: {train_loss:.4f}, Val loss: {val_loss:.4f}")   
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("Early stopping")
        break
    
early_stopping.load_best_model(model)

model.eval()
correct = 0
total = 0
all_preds = []
all_targets = []

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum.item()
        
        all_preds.extend(predicted.cpu().numpy())
        all_targets.extend(target.cpu().numpy())
        
            
accuracy = 100 * correct / total
precision = precision_score(all_targets, all_preds, average='weighted')
recall = recall_score(all_targets, all_preds, average='weighted')
f1 = f1_score(all_targets, all_preds, average='weighted')

# Print the results
print(f'Accuracy: {accuracy:.2f}%')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')

torch.save(model.state_dict(), "./braincnn_prototype.weights")
